# Opprette tabluar data asset (MLTable) i SDK v2

[MLTable](https://learn.microsoft.com/en-us/azure/machine-learning/concept-data#mltable) er Azure ML Studio sitt forsøk på å forenkle konsumering av tabluære data. MLTable erstatter det som heter TabularDataset i V1. Målet er å abstrahere vekk mest mulig ansvar for definisjonen av datasettet vekk fra consumenten og flytte det til der dataene ligger.

Noen features fra MLTable:
- Lazy loading av data
- Definere schema der data ligger
- Filtrere hvilke filer som er relevante
- Filtrere vekk kolonner
- Enkle datatransformasjoner

Vi begynner med å hente MNIST fra azureml sin opendatasets pakke og fjerner kolonne for radnummer.

In [ ]:
from azureml.opendatasets import MNIST
mnist_df = MNIST.get_tabular_dataset().to_pandas_dataframe()
mnist_df.head()

Vi gjør en random 20/80 split og lagrer lokalt som csv.

In [ ]:
from sklearn.model_selection import train_test_split
train_data_folder = os.path.join(os.getcwd() + '/data/train')
test_data_folder = os.path.join(os.getcwd() + '/data/test')

os.makedirs(train_data_folder, exist_ok=True)
os.makedirs(test_data_folder, exist_ok=True)

train_data_df, test_data_df = train_test_split(mnist_df, train_size=0.8, shuffle=True)
train_data_df.to_csv(train_data_folder + '/MNIST_Train.csv', index=False)
test_data_df.to_csv(test_data_folder + '/MNIST_Test.csv', index=False)

In [ ]:
print(train_data_df.shape)
print(test_data_df.shape)

Oppretter [MLTable](https://learn.microsoft.com/en-us/azure/machine-learning/concept-data#mltable) filer som ligger sammen med sine respektive datasett.


In [ ]:
%%writefile data/train/MLTable
type: mltable

paths:
  - pattern: ./*.csv
transformations:
  - read_delimited:
      delimiter: ','
      encoding: ascii
      header: all_files_same_headers

In [ ]:
%%writefile data/test/MLTable
type: mltable

paths:
  - pattern: ./*.csv
transformations:
  - read_delimited:
      delimiter: ','
      encoding: ascii
      header: all_files_same_headers

Deretter brukes Azure ML SDKv2 for å registrere datasettet.

In [ ]:
%pip install azure-ai-ml

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id = "INSERT_SUBSCRIPTION_ID",
    resource_group_name = "INSERT_RESOURCE_GROUP",
    workspace_name = "INSERT_WORKSPACE",
)

train_data = Data(
    path='./data/train',
    type=AssetTypes.MLTABLE,
    description="",
    name="MNIST_Train",
    version='1'
)

ml_client.data.create_or_update(train_data)

test_data = Data(
    path='./data/test',
    type=AssetTypes.MLTABLE,
    description="",
    name="MNIST_Test",
    version='1',
)

ml_client.data.create_or_update(test_data)

We load the mltable dataset like this when it is received in a component receiving the data.

In [ ]:
# Re-import dataset in a component

import mltable

tbl_test = mltable.load("data/test")
df_test = tbl_test.to_pandas_dataframe()

X_test_flattened = df.drop("label", axis=1).to_numpy().astype('uint8')
X_test = X_test_flattened.reshape(X_test_flattened.shape[0], 28, 28)
y_test = df["label"].to_numpy().astype('uint8')
X_train